# Lip Reading Model Setup

This notebook sets up the lip reading models and dependencies for Google Colab.


In [ ]:
# Clone repository and setup dependencies
import os
import subprocess
from pathlib import Path

def run(cmd, desc=""):
    """Run shell command"""
    print(f"\n{'='*60}")
    print(f"{desc}")
    print(f"{'='*60}")
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print(f"❌ Error: {result.stderr}")
        raise RuntimeError(f"Command failed: {cmd}")
    print(result.stdout)

# Setup repository
REPO_DIR = "/content/lipreading"

if not os.path.exists(REPO_DIR):
    run("git clone https://github.com/your-repo/lipreading.git /content/lipreading", "Cloning repository")

os.chdir(REPO_DIR)
print(f"✅ Working directory: {os.getcwd()}")


In [ ]:
# Install dependencies
run("pip install -r requirements.txt", "Installing base dependencies")
run("pip install -r requirements-web.txt", "Installing web dependencies")
print("✅ Dependencies installed!")


In [ ]:
# Download models from Google Drive
import gdown
import shutil

# Google Drive IDs
MAIN_MODEL_GDRIVE_ID = "1tgzkAHkDcDf3vijkG9PZOxmwUz_YBuCf"
LM_MODEL_GDRIVE_ID = "1eJZ5n4GU2C3i0WnX3cogK-MVUHz4gk-l"
VIDEO_SAMPLES_GDRIVE_ID = "your-video-samples-id"  # Add your video samples ID

def download_from_gdrive(file_id, target_dir, desc="", file_types=None):
    """Download folder from Google Drive and move specified files to correct location"""
    print(f"\n📥 Downloading {desc} from Google Drive...")
    os.makedirs(target_dir, exist_ok=True)

    # Download folder from Google Drive
    cmd = f"gdown --folder --id {file_id} -O {target_dir}"
    run(cmd, f"Downloading {desc}")

    # Default to model files if no file types specified
    if file_types is None:
        file_types = ['model.pth', 'model.json']

    # Find and move specified files from subdirectories to target_dir
    for root, dirs, files in os.walk(target_dir):
        for file in files:
            # Check if file matches any of the specified types
            if any(file.endswith(ext) for ext in file_types) or file in file_types:
                src = os.path.join(root, file)
                dst = os.path.join(target_dir, file)
                if src != dst:
                    print(f"Moving {file} to correct location")
                    shutil.move(src, dst)

    # Clean up empty subdirectories
    for root, dirs, files in os.walk(target_dir, topdown=False):
        for dir_name in dirs:
            dir_path = os.path.join(root, dir_name)
            if not os.listdir(dir_path):  # If directory is empty
                os.rmdir(dir_path)
                print(f"Cleaned up empty directory: {dir_name}")

    print(f"✅ {desc} setup complete")

# Download models
download_from_gdrive(MAIN_MODEL_GDRIVE_ID, 
                    "/content/lipreading/benchmarks/LRS3/models/LRS3_V_WER19.1", 
                    "LRS3_V_WER19.1 model")

download_from_gdrive(LM_MODEL_GDRIVE_ID, 
                    "/content/lipreading/benchmarks/LRS3/language_models/lm_en_subword", 
                    "lm_en_subword language model")

# Optional: Download video samples
if VIDEO_SAMPLES_GDRIVE_ID != "your-video-samples-id":
    download_from_gdrive(VIDEO_SAMPLES_GDRIVE_ID, 
                        "/content/lipreading/video_samples", 
                        "video samples",
                        ['.mp4', '.avi', '.mov'])

print("\n✅ All models downloaded and ready!")


In [ ]:
# Start web server
print("🚀 Starting Lip Reading Web Server...")
print("This will create a public URL accessible from anywhere!")

# Change to web directory and run app
os.chdir("/content/lipreading")
exec(open('web/app.py').read())
